# M1M3 position data
Craig Lage - 16-Apr-23 \
Comparing different position measurements.

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from astropy.time import Time, TimeDelta
import lsst.ts.cRIOpy.M1M3FATable as M1M3FATable
from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('usdf_efd')
FATABLE = M1M3FATable.FATABLE

In [ ]:
start = Time("2023-07-31T19:08:00", scale='utc')
raised = Time("2023-07-31T19:26:00", scale='utc')
end = Time("2023-07-31T19:28:00", scale='utc') 
hardpoint_data = await client.select_time_series("lsst.sal.MTM1M3.hardpointActuatorData", "*", start, end)
monitor_data = await client.select_time_series("lsst.sal.MTM1M3.hardpointMonitorData", "*", start, end)
ims_data = await client.select_time_series("lsst.sal.MTM1M3.imsData", "*", start, end)
force_data = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", \
                                             ["primaryCylinderForce0", "timestamp"], start, end)
timestamp = start.isot.split('.')[0].replace('-','').replace(':','')

In [ ]:
plots = ['xPosition', 'yPosition', 'zPosition', 'xRotation', 'yRotation', 'zRotation']

fig, axs = plt.subplots(3,2,figsize=(8,10))
plt.subplots_adjust(hspace=0.6, wspace=1.0)
plt.suptitle(f"Mirror position changes during raise: {timestamp}", fontsize=16)
plot_counter = 0
for i in range(3):
    for j in range(2):
        
        smoothed_ims_data = ims_data[plots[plot_counter]].rolling(10).mean()
        smoothed_ims_data = smoothed_ims_data.dropna()
        smoothed_ims_data -= smoothed_ims_data[0]
        smoothed_hardpoint_data = hardpoint_data[plots[plot_counter]].rolling(10).mean()
        
        if plot_counter > 2:
            smoothed_ims_data *= 1E6
            smoothed_hardpoint_data *= 1E6
            unit = 'microDeg'
        else:
            smoothed_ims_data *= 1E6
            smoothed_hardpoint_data *= 1E6
            unit = 'um'
        axs[i][j].set_title(plots[plot_counter])
        axs[i][j].set_ylabel(f'HP Displacement({unit})')
        axs[i][j].yaxis.label.set_color('blue')
        ax = axs[i][j].twinx()
        ax.set_ylabel(f'IMS ({unit})')
        ax.yaxis.label.set_color('red')
        max_pos = np.median(smoothed_ims_data.values.max())
        min_pos = np.median(smoothed_ims_data.values.min())
        pos_delta = max_pos - min_pos
        axs[i][j].set_title(plots[plot_counter]+f"\n Position Delta = {pos_delta:.1f} {unit}")

        smoothed_hardpoint_data.plot(ax=axs[i][j], color='blue', label='Hardpoint')
        smoothed_ims_data.plot(ax=ax, color='red', ls='--', label='IMS')
        hp_values = smoothed_hardpoint_data.values
        hp_values = hp_values[~np.isnan(hp_values)]
        y1 = hp_values.min() + 0.2 * (hp_values.max() - hp_values.min())
        y2 = hp_values.min() + 0.8 * (hp_values.max() - hp_values.min())
        axs[i][j].axvline(start.isot, ls = '--', color='orange')
        axs[i][j].text(start.isot, y1, "Lowered", color='orange')
        axs[i][j].axvline(end.isot, ls = '--', color='black')
        axs[i][j].text(end.isot, y2, " Raised", color='black')
        plot_counter += 1
plt.savefig(f"/home/c/cslage/u/MTM1M3/data/Position_Raise_{timestamp}.png")

In [ ]:
HP_disps = []
LVDTs = []
for i in range(6):
    hp_disp_raised = hardpoint_data[hardpoint_data['timestamp'] > raised.unix_tai][f'displacement{i}'][0:100].mean()
    hp_disp_lowered =  hardpoint_data[f'displacement{i}'][0:100].mean()
    hp_delta = (hp_disp_raised - hp_disp_lowered) * 1.0E6
    
    lv_disp_raised = monitor_data[monitor_data['timestamp'] > raised.unix_tai][f'displacementLVDT{i}'][0:100].mean()
    lv_disp_lowered =  monitor_data[f'displacementLVDT{i}'][-100:-1].mean()
    lv_delta = (lv_disp_raised - lv_disp_lowered)
    #if i not in [4]:
    HP_disps.append(hp_delta)
    LVDTs.append(lv_delta)
    plt.text(hp_delta-20.0, lv_delta, str(i))
    print(i, hp_delta, lv_delta)
plt.title(f"Mirror displacement comparisons \n during raising: {timestamp}", fontsize=16)
plt.xlabel("hardpointActuatorData.displacementx (um)")
plt.ylabel("hardpointMonitorData.displacementLVDTx (??)")
plt.scatter(HP_disps, LVDTs)
plt.savefig(f"/home/c/cslage/u/MTM1M3/data/HP_Displacements_Comparisons_{timestamp}.png")

In [ ]:
timestamp = start.isot.split('.')[0].replace('-','').replace(':','')
pdf = PdfPages(f"/home/c/cslage/u/MTM1M3/data/Mirror_Raise_{timestamp}.pdf")

HP_disps = []
LVDTs = []
HP_pos = []
ims_pos = []
for i in range(6):
    fig, axs = plt.subplots(5,1,figsize=(8,12))
    plt.subplots_adjust(hspace=1.0)
    plt.suptitle(f"Mirror position changes during lowering: {timestamp}", fontsize=16)
    axs[0].set_title("primaryCylinderForce0")
    axs[0].set_ylabel("N")
    force_data["primaryCylinderForce0"].plot(ax=axs[0])
    
    y1 = 100.0
    y2 = 600.0
    axs[0].axvline(raised.isot, ls = '--', color='orange')
    axs[0].text(raised.isot, y1, " Raised", color='orange')
    axs[0].axvline(end.isot, ls = '--', color='black')
    axs[0].text(raised.isot, y2, "                           Lowered", color='black')
  
    axs[1].set_title("IMS zPosition")
    axs[1].set_ylabel("m")
    ims_data["zPosition"].plot(ax=axs[1])                             
    axs[2].set_title("Hardpoint encoder zPosition")
    axs[2].set_ylabel("m")
    hardpoint_data["zPosition"].plot(ax=axs[2])
    axs[3].set_title(f"Hardpoint Displacement LVDT{i}")
    axs[3].set_ylabel("??")
    monitor_data[f"displacementLVDT{i}"].plot(ax=axs[3])
    axs[4].set_title(f"Hardpoint Breakaway LVDT{i}")
    axs[4].set_ylabel("??")
    monitor_data.plot.scatter(ax=axs[4], x='timestamp', y=f"breakawayLVDT{i}")
    pdf.savefig(fig)  # saves the current figure into a pdf page
    plt.clf()
pdf.close()

In [ ]:
posNames = ['xPosition', 'yPosition', 'zPosition', 'xRotation', 'yRotation', 'zRotation']
fig = plt.figure(figsize=(10,5))
plt.subplots_adjust(wspace = 0.5)
plt.suptitle(f"Mirror position comparisons during raising: {timestamp}", fontsize=16)                 

for i in range(2):
    IMS = []
    HP = []
    plt.subplot(1,2,i+1)
    for j in range(3):
        posName = posNames[int(3 * i + j)]
        ims_raised = ims_data[ims_data['timestamp'] > raised.unix_tai][posName][0:100].mean()
        ims_lowered =  ims_data[posName][-100:-1].mean()
        ims_delta = (ims_raised - ims_lowered) * 1.0E6

        hp_raised = hardpoint_data[hardpoint_data['timestamp'] > raised.unix_tai][posName][0:100].mean()
        hp_lowered =  hardpoint_data[posName][-100:-1].mean()
        hp_delta = (hp_raised - hp_lowered) * 1.0E6
        HP.append(hp_delta)
        IMS.append(ims_delta)
        if j == 2:
            shift = -1500.0
        else:
            shift = 200.0
        plt.text(hp_delta+shift, ims_delta, posName)
        print(posName, ims_delta, hp_delta, ims_delta/hp_delta)
    
    if i == 0:
        plt.title("Positions")
        plt.xlabel("Hardpoint position (um)")
        plt.ylabel("IMS position (um)")
        xs = np.linspace(0,5000, 500)
        plt.plot(xs, xs, ls='--', color='red')
        
    else:
        plt.title("Rotations")
        plt.xlabel("Hardpoint rotation (microdeg)")
        plt.ylabel("IMS rotation (microdeg)")
        xs = np.linspace(-7500, -1000, 500)
        plt.plot(xs, xs, ls='--', color='red')
    
    plt.scatter(HP, IMS)  
plt.savefig(f"/home/c/cslage/u/MTM1M3/data/HP_Position_Comparisons_{timestamp}.png")

In [ ]:
start = Time("2023-07-31T19:17:00", scale='utc')
end = Time("2023-07-31T19:18:15", scale='utc') 
hardpoint_data = await client.select_time_series("lsst.sal.MTM1M3.hardpointActuatorData", "*", start, end)
monitor_data = await client.select_time_series("lsst.sal.MTM1M3.hardpointMonitorData", "*", start, end)

HP_disps = []
LVDTs = []
HP_pos = []
ims_pos = []
for i in [3]:
    fig, axs = plt.subplots(3,1,figsize=(8,12))
    plt.subplots_adjust(hspace=1.0)
    plt.suptitle(f"Mirror position changes during raising", fontsize=16)

    axs[0].set_title("Hardpoint encoder zPosition")
    axs[0].set_ylabel("m")
    hardpoint_data["zPosition"].plot(ax=axs[0])
    axs[1].set_title(f"Hardpoint Displacement LVDT{i}")
    axs[1].set_ylabel("??")
    monitor_data[f"displacementLVDT{i}"].plot(ax=axs[1])
    disp = -hardpoint_data[f"displacement{i}"].values * 1.0E3
    lvdt = monitor_data[f"displacementLVDT{i}"].values
    axs[2].scatter(disp, lvdt)
    fit = np.polyfit(disp, lvdt, 1)
    xplot = np.linspace(disp.min(), disp.max(), 1000)
    yplot = np.polyval(fit, xplot)
    axs[2].plot(xplot, yplot, ls='--', color='red')
    multiplier = 1.0 / fit[0]
    axs[2].text(disp.min(), (lvdt.min() + lvdt.max()) / 2.0, f"LVDT mutiplier = \n{multiplier:.4f} to convert to mm.")
    


In [ ]:
start = Time("2023-07-31T19:17:00", scale='utc')
end = Time("2023-07-31T19:18:15", scale='utc') 
hardpoint_data = await client.select_time_series("lsst.sal.MTM1M3.hardpointActuatorData", "*", start, end)
monitor_data = await client.select_time_series("lsst.sal.MTM1M3.hardpointMonitorData", "*", start, end)

fig, axs = plt.subplots(3,2,figsize=(8,10))
plt.subplots_adjust(hspace=0.5, wspace=0.5)
plt.suptitle(f"Hardpoint LVDT calibrations", fontsize=16)
plot_counter = 0
for i in range(3):
    for j in range(2):
        disp = -hardpoint_data[f"displacement{plot_counter}"].values * 1.0E3
        lvdt = monitor_data[f"displacementLVDT{plot_counter}"].values
        axs[i][j].set_title(f"Hardpoint {plot_counter}")
        axs[i][j].set_xlabel(f"Hardpoint Displacement{plot_counter} (mm)")
        axs[i][j].set_ylabel(f"LVDT Displacement{plot_counter}")
        axs[i][j].scatter(disp, lvdt)
        fit = np.polyfit(disp, lvdt, 1)
        xplot = np.linspace(disp.min(), disp.max(), 1000)
        yplot = np.polyval(fit, xplot)
        axs[i][j].plot(xplot, yplot, ls='--', lw=3, color='red')
        multiplier = 1.0 / fit[0]
        axs[i][j].text(disp.min(), lvdt.max() - (lvdt.max() - lvdt.min()) * 0.2, \
                       f"LVDT mutiplier = {multiplier:.4f}\n to convert to mm.")
        plot_counter += 1
plt.savefig(f"/home/c/cslage/u/MTM1M3/data/Hardpoint_LVDT_Calibrations_31Jul23.png")